# Introduction

In this Jupyter notebook, we will explore how to evaluate and mitigate bias in machine learning models using the Python package DALEX.

We will use a diabetes prediction dataset and build a decision tree, random forest, and logistic regression models to predict the presence of diabetes.

Then, we will evaluate the models' fairness using different fairness metrics and techniques, including measuring parity loss, visualizing fairness metrics, and using resampling methods to mitigate bias.

Finally, we will compare the fairness of the original and mitigated models and assess their trade-offs.


In [18]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Import dataset
dataset = pd.read_csv('diabetes.csv')


# Models

This code defines three pipelines using different machine learning algorithms - Decision Tree, Random Forest, and Logistic Regression - to classify the outcome of diabetes based on various features such as age, pregnancies, glucose level, blood pressure, skin thickness, insulin level, BMI, and diabetes pedigree function.

First we split the dataset into training and testing sets using the train_test_split function from scikit-learn. Then, it defines the transformers for preprocessing the data using the ColumnTransformer class from scikit-learn. The numerical transformer applies standard scaling to the numerical features, while the categorical transformer applies one-hot encoding to the categorical feature "AgeCategory". The ColumnTransformer combines these two transformers and applies them to the corresponding features.

Next, the code defines three pipelines using the different machine learning algorithms mentioned earlier. Each pipeline includes the preprocessor defined earlier and the corresponding classifier. The Decision Tree classifier has a maximum depth of 7, while the Random Forest classifier has 200 estimators and a maximum depth of 7. The Logistic Regression classifier uses the default hyperparameters.

Finally, the code fits each pipeline to the training data using the fit method of each pipeline.


In [19]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(dataset.drop(
    'Outcome', axis=1), dataset['Outcome'], test_size=0.3, random_state=100)

categorical_features = ['AgeCategory']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_features = ['Pregnancies', 'Glucose', 'BloodPressure',
                      'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Decision Tree
dt = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', DecisionTreeClassifier(max_depth=7, random_state=100))])

dt.fit(X_train, y_train)

# Random Forest
rf = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', RandomForestClassifier(n_estimators=200, max_depth=7, random_state=100))])

rf.fit(X_train, y_train)

# Logistic Regression
lr = Pipeline(steps=[('preprocessor', preprocessor),
                     ('classifier', LogisticRegression())])

lr.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Pregnancies', 'Glucose',
                                                   'BloodPressure',
                                                   'SkinThickness', 'Insulin',
                                                   'BMI',
                                                   'DiabetesPedigreeFunction',
                                                   'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['AgeCategory'])])),
                ('classifier', LogisticRegression())])

# Explainers

To explain the predictions made by the three classifiers, we use the DALEX library. Specifically, we create an Explainer object for each of the three classifiers, which allows visualizing and analyze the models in more detail.


In [20]:
import dalex as dx

exp_tree = dx.Explainer(dt, X_test, y_test, label='Decision Tree')
exp_forest = dx.Explainer(rf, X_test, y_test, label='Random Forest')
exp_logreg = dx.Explainer(lr, X_test, y_test, label='Logistic Regression')

Preparation of a new explainer is initiated

  -> data              : 229 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 229 values
  -> model_class       : sklearn.tree._classes.DecisionTreeClassifier (default)
  -> label             : Decision Tree
  -> predict function  : <function yhat_proba_default at 0x000001D7CBD78940> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0, mean = 0.39, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = 0.00727, max = 1.0
  -> model_info        : package sklearn

A new explainer has been created!
Preparation of a new explainer is initiated

  -> data              : 229 rows 10 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converte

# Model Performance Measures

This code outputs a table that shows the performance metrics of the three classifiers.
In particular the following metrics are calculated:
- Recall: the proportion of positive cases that were correctly identified by the classifier (also known as sensitivity or true positive rate)
- Precision: the proportion of positive predictions that were correctly identified by the classifier (also known as positive predictive value)
- F1: the harmonic mean of recall and precision, which provides a balanced measure of performance that takes into account both false positives and false negatives
- Accuracy: the proportion of all predictions that were correct (i.e., the number of true positives and true negatives divided by the total number of predictions)
- auc: the area under the receiver operating characteristic (ROC) curve, which provides a measure of how well the classifier can distinguish between positive and negative cases

In [21]:
pd.concat([exp.model_performance().result for exp in [
          exp_logreg, exp_tree, exp_forest]])

,recall,precision,f1,accuracy,auc
Logistic Regression,0.560440,0.718310,0.629630,0.737991,0.827441
Decision Tree,0.582418,0.638554,0.609195,0.703057,0.693423
Random Forest,0.648352,0.746835,0.694118,0.772926,0.835244


# Variable Importance

This code cell generates a variable importance plot for the Decision Tree model, comparing it to the variable importance plots for the Random Forest and Logistic Regression models.

The `model_parts()` method computes the contributions of each variable to the model prediction for the three models


In [22]:
exp_tree.model_parts().plot(
    objects=[exp_forest.model_parts(), exp_logreg.model_parts()])

# Fairness: Age

We define a binary protected attribute based on the age attribute. We set the age threshold to 50, and define all individuals with an age greater than 50 as "old" and all others as "young".

We also define "old" as the privileged group.

We then use the `model_fairness()` method from DALEX to compute fairness metrics for each model, based on the protected attribute and privileged group defined above.
The results are `FairnessResult` objects that store various fairness metrics:


- True Positive Rate (**TPR**): the proportion of positive cases that were correctly identified by the classifier (also known as sensitivity or recall)
- Accuracy (**ACC**): the proportion of all predictions that were correct (i.e., the number of true positives and true negatives divided by the total number of predictions)
- Positive Predictive Value (**PPV**): the proportion of positive predictions that were correctly identified by the classifier (also known as precision)
- False Positive Rate (**FPR**): the proportion of negative cases that were incorrectly identified by the classifier (also known as fall-out)
- Statistical Parity (**STP**): the difference between the probability of a positive outcome for the privileged group and the probability of a positive outcome for the unprivileged group


 Metrics are calculated for each subgroup, and then their scores are divided by the score of the privileged subgroup

In [23]:
protected = np.where(X_test.Age > 50, 'old', 'young')
privileged = 'old'

mf_tree = exp_tree.model_fairness(protected=protected,
                                  privileged=privileged)
mf_forest = exp_forest.model_fairness(protected=protected,
                                      privileged=privileged)
mf_logreg = exp_logreg.model_fairness(protected=protected,
                                      privileged=privileged)


In [24]:
mf_tree.fairness_check()


Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV       FPR    STP
young  0.938211  1.110937  0.950525  0.618619  0.725


In [25]:
mf_forest.fairness_check()


Bias detected in 1 metric: STP

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR     ACC       PPV       FPR       STP
young  0.816645  0.9625  0.877551  0.856287  0.683333


In [26]:
mf_logreg.fairness_check()


Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV       FPR       STP
young  0.606383  1.027778  1.056686  0.285372  0.421875


### Visualize the result


In [27]:
mf_tree.plot(objects=[mf_forest, mf_logreg])


We evaluate the cumulated parity representing the previous metrics in a stacked bar plot. The cumulated parity is the sum of the absolute values of the differences between the privileged and unprivileged groups for each metric. The cumulated parity is a measure of the overall fairness of the model.


In [28]:
mf_tree.plot(objects=[mf_logreg, mf_forest], type='stacked')

# Mitigation

 In this section, we will use the Resampling technique provided by the dalex.fairness package to mitigate fairness issues. This technique implements the resampling technique for mitigation as presented in this paper: https://link.springer.com/content/pdf/10.1007/s10115-011-0463-8.pdf.

## Resampling

We will apply the resampling technique in two ways: uniform and preferential.

In uniform resampling, DALEX will randomly select a subset of the majority group to balance the class distribution.

In preferential resampling, DALEX will randomly select a subset of the minority group and oversample it to balance the class distribution.

In [29]:
from dalex.fairness import resample
from copy import copy

#decision tree

clf_u_dt = copy(dt)
clf_p_dt = copy(dt)

#random forest 

clf_u_rf = copy(rf)
clf_p_rf = copy(rf)

#logistic regression

clf_u_lr = copy(lr)
clf_p_lr = copy(lr)

In [30]:
# resample on decision tree, we include the preferential resampling of the decision tree for completeness, 
#although without any prio it's not really useful

indices_uniform = resample(protected, y_test, verbose=False)
indices_preferential = resample(protected,
                                y_test,
                                type='preferential',  # different type
                                probs=exp_tree.y_hat,  # requires probabilities
                                verbose=False)

#decision tree
clf_u_dt.fit(X_train.iloc[indices_uniform, :], y_train.iloc[indices_uniform])
clf_p_dt.fit(X_train.iloc[indices_preferential, :], y_train.iloc[indices_preferential])

#random forest
clf_u_rf.fit(X_train.iloc[indices_uniform, :], y_train.iloc[indices_uniform])

#logistic regression
clf_u_lr.fit(X_train.iloc[indices_uniform, :], y_train.iloc[indices_uniform])


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['Pregnancies', 'Glucose',
                                                   'BloodPressure',
                                                   'SkinThickness', 'Insulin',
                                                   'BMI',
                                                   'DiabetesPedigreeFunction',
                                                   'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['AgeCategory'])])),
                ('classifier', LogisticRegression())])

Now we can create the Explainers for the models using the resampled data. And as we did before, we can compute the fairness metrics for the two models using the `model_fairness` function of `dalex.Explainer`.

The protected and privileged variables are set to the same values as before.

In [31]:
#decision tree explainers
exp1_dt = dx.Explainer(clf_u_dt, X_test, y_test, verbose = False)
exp2_dt = dx.Explainer(clf_p_dt, X_test, y_test, verbose = False)

#random forest expaliner
exp_rf = dx.Explainer(clf_u_rf, X_test, y_test, verbose = False)

#logistic regression explainer
exp_lr = dx.Explainer(clf_u_lr, X_test, y_test, verbose = False)

mf_u_dt = exp1_dt.model_fairness(protected=protected, privileged=privileged, label="Decision Tree u")#uniform
mf_p_dt = exp2_dt.model_fairness(protected=protected, privileged=privileged, label="Decision Tree p")#preferential

mf_u_rf = exp_rf.model_fairness(protected=protected, privileged=privileged, label="Random Forest")
mf_u_lr = exp_lr.model_fairness(protected=protected, privileged=privileged, label="Logistic Regression")

mf_u_dt.plot([mf_u_rf, mf_u_lr, mf_p_dt,])

In [32]:
#decision tree fairness check
mf_u_dt.fairness_check()
mf_p_dt.fairness_check()



Bias detected in 2 metrics: TPR, ACC

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV   FPR       STP
young  1.746753  1.309615  1.036778  0.92  1.242857
Bias detected in 2 metrics: TPR, ACC

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV   FPR       STP
young  1.746753  1.309615  1.036778  0.92  1.242857


In [33]:
#random forest fairness check
mf_u_rf.fairness_check()

Bias detected in 2 metrics: TPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR       ACC       PPV       FPR       STP
young  0.760116  0.967105  0.864303  0.808383  0.645455


In [34]:
#logistic regression fairness check
mf_u_lr.fairness_check()

Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'old'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
            TPR     ACC       PPV       FPR       STP
young  0.682033  1.0625  1.090116  0.285372  0.459375
